In [ ]:
!pip install gensim
!pip install pyLDAvis

In [27]:
# -*- coding: utf-8 -*-
"""
Created on Mon Aug 19 12:16:43 2019

@author: isswan
"""

# In this workshop we perform topic modeling using gensim
# The "Class" labels here are only used for sanity check of the topics discovered later.
# Remember, in actual use of topic modelling, the documents DON'T come with labeled classes.
# It's unsupervised learning.

import numpy as np
import pandas as pd

import nltk
from nltk import FreqDist, SnowballStemmer
import string
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from google.colab import drive
drive.mount('/content/gdrive')
base_path    = "/content/gdrive/My Drive/Colab Notebooks/4/"

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
  # Searched in:
  #   - '/root/nltk_data'
  #   - '/usr/share/nltk_data'
  #   - '/usr/local/share/nltk_data'
  #   - '/usr/lib/nltk_data'
  #   - '/usr/local/lib/nltk_data'
  #   - '/usr/nltk_data'
  #   - '/usr/lib/nltk_data'

In [28]:
news=pd.read_table(base_path+'MovieStories.utf8.txt',header=0)
mystopwords=stopwords.words("english") + ['life','film','movie','one','two']
WNlemma = nltk.WordNetLemmatizer()

In [29]:
def pre_process(text):
    tokens = nltk.word_tokenize(text)
    tokens=[ WNlemma.lemmatize(t.lower()) for t in tokens]
    tokens=[ t for t in tokens if t not in mystopwords]
    tokens = [ t for t in tokens if len(t) >= 3 ]
    return(tokens)


In [30]:
text = news['Storyline']
toks = text.apply(pre_process)
toks
# toks is now a DF where the obly field 'Storyline' becomes a list of tokens

0       [action-packed, drama, christian, high, school...
1       [holiday, comedy, centered, around, neighbor, ...
2       [day, get, word, 'll, home, week, group, soldi...
3       [story, doogal, adorable, candy-loving, mutt, ...
4       [drama, focus, period, mary, joseph, journeyed...
                              ...                        
1048    [futuristic, twist, robert, louis, stevenson, ...
1049    [ten, year, 'phantom, menace, threatened, plan...
1050    [agent, another, day, office, monitoring, lice...
1051    [frida, chronicle, frida, kahlo, shared, unfli...
1052    [story, detective, agustin, rejas, man, clingi...
Name: Storyline, Length: 1053, dtype: object

In [31]:
# Use dictionary (built from corpus) to prepare a DTM (using frequency)
import logging

import gensim 
from gensim import corpora

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Filter off any words with document frequency less than 2, or appearing in more than 90% documents
dictionary = corpora.Dictionary(toks)
print(dictionary)




2021-07-18 02:21:04,770 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-07-18 02:21:04,845 : INFO : built Dictionary(12151 unique tokens: ['action-packed', 'battle', 'believe', 'brooke', 'challenge']...) from 1053 documents (total 54912 corpus positions)


Dictionary(12151 unique tokens: ['action-packed', 'battle', 'believe', 'brooke', 'challenge']...)


In [32]:
dictionary.filter_extremes(no_below=2, no_above=0.8)
"""
        no_below : int, optional
            Keep tokens which are contained in at least `no_below` documents.
        no_above : float, optional
            Keep tokens which are contained in no more than `no_above` documents
            (fraction of total corpus size, not an absolute number).
"""
print(dictionary)

2021-07-18 02:21:09,310 : INFO : discarding 6862 tokens: [('cry', 1), ('dismissed', 1), ('infertility', 1), ('organizing', 1), ('shiloh', 1), ('transpires', 1), ('undying', 1), ('decorate', 1), ('visible', 1), ('dislocation', 1)]...
2021-07-18 02:21:09,311 : INFO : keeping 5289 tokens which were in no less than 2 and no more than 842 (=80.0%) documents
2021-07-18 02:21:09,322 : INFO : resulting dictionary: Dictionary(5289 unique tokens: ['action-packed', 'battle', 'believe', 'brooke', 'challenge']...)


Dictionary(5289 unique tokens: ['action-packed', 'battle', 'believe', 'brooke', 'challenge']...)


In [33]:
#dtm here is a list of lists, which is exactly a matrix
dtm = [dictionary.doc2bow(d) for d in toks]
print(dtm)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 2), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 2), (23, 1), (24, 1), (25, 2), (26, 1), (27, 2), (28, 3), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1)], [(61, 1), (62, 1), (63, 2), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1)], [(30, 2), (68, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 2), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 2), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 2), (106, 1), (107, 1

In [34]:
lda = gensim.models.ldamodel.LdaModel(dtm, num_topics = 5, 
                                      id2word = dictionary, 
                                      passes=10,chunksize=128,random_state=10)



2021-07-18 02:21:15,865 : INFO : using symmetric alpha at 0.2
2021-07-18 02:21:15,866 : INFO : using symmetric eta at 0.2
2021-07-18 02:21:15,869 : INFO : using serial LDA version on this node
2021-07-18 02:21:15,876 : INFO : running online (multi-pass) LDA training, 5 topics, 10 passes over the supplied corpus of 1053 documents, updating model once every 128 documents, evaluating perplexity every 1053 documents, iterating 50x with a convergence threshold of 0.001000
2021-07-18 02:21:15,878 : INFO : PROGRESS: pass 0, at document #128/1053
2021-07-18 02:21:15,977 : INFO : merging changes from 128 documents into a model of 1053 documents
2021-07-18 02:21:15,984 : INFO : topic #0 (0.200): 0.010*"find" + 0.007*"new" + 0.006*"house" + 0.006*"game" + 0.006*"friend" + 0.005*"family" + 0.005*"come" + 0.005*"american" + 0.004*"n't" + 0.004*"home"
2021-07-18 02:21:15,986 : INFO : topic #1 (0.200): 0.008*"friend" + 0.007*"find" + 0.006*"day" + 0.005*"home" + 0.005*"get" + 0.005*"love" + 0.005*"wa

[(0,
  '0.010*"set" + 0.008*"agent" + 0.006*"force" + 0.005*"army" + 0.005*"escape" + 0.005*"must" + 0.004*"named" + 0.004*"battle" + 0.004*"prove" + 0.004*"crew"'),
 (1,
  '0.011*"family" + 0.010*"new" + 0.009*"find" + 0.009*"get" + 0.009*"day" + 0.009*"friend" + 0.008*"n\'t" + 0.008*"come" + 0.007*"home" + 0.007*"love"'),
 (2,
  '0.012*"world" + 0.010*"vampire" + 0.008*"story" + 0.007*"american" + 0.007*"america" + 0.006*"love" + 0.006*"sam" + 0.006*"young" + 0.006*"human" + 0.005*"situation"'),
 (3,
  '0.010*"new" + 0.009*"school" + 0.008*"get" + 0.008*"friend" + 0.007*"going" + 0.007*"want" + 0.006*"high" + 0.005*"drug" + 0.005*"york" + 0.005*"..."'),
 (4,
  '0.007*"man" + 0.006*"find" + 0.006*"daughter" + 0.006*"wife" + 0.006*"team" + 0.006*"father" + 0.005*"year" + 0.005*"child" + 0.005*"death" + 0.005*"family"')]

In [35]:
lda.show_topics(10)

[(0,
  '0.010*"set" + 0.008*"agent" + 0.006*"force" + 0.005*"army" + 0.005*"escape" + 0.005*"must" + 0.004*"named" + 0.004*"battle" + 0.004*"prove" + 0.004*"crew"'),
 (1,
  '0.011*"family" + 0.010*"new" + 0.009*"find" + 0.009*"get" + 0.009*"day" + 0.009*"friend" + 0.008*"n\'t" + 0.008*"come" + 0.007*"home" + 0.007*"love"'),
 (2,
  '0.012*"world" + 0.010*"vampire" + 0.008*"story" + 0.007*"american" + 0.007*"america" + 0.006*"love" + 0.006*"sam" + 0.006*"young" + 0.006*"human" + 0.005*"situation"'),
 (3,
  '0.010*"new" + 0.009*"school" + 0.008*"get" + 0.008*"friend" + 0.007*"going" + 0.007*"want" + 0.006*"high" + 0.005*"drug" + 0.005*"york" + 0.005*"..."'),
 (4,
  '0.007*"man" + 0.006*"find" + 0.006*"daughter" + 0.006*"wife" + 0.006*"team" + 0.006*"father" + 0.005*"year" + 0.005*"child" + 0.005*"death" + 0.005*"family"')]

In [ ]:

##Evaluate the coherence score of LDA models
'''
u_mass:prefer the model close to 0 
c_v: [0,1], prefer bigger value   
Do not fully rely on the coherence score
'''
from gensim.models.coherencemodel import CoherenceModel
cm_umass = CoherenceModel(lda,  dictionary=dictionary, corpus=dtm, coherence='u_mass')
cm_cv = CoherenceModel(lda,  dictionary=dictionary, texts=toks, coherence='c_v')
lda_umass = cm_umass.get_coherence()
lda_cv = cm_cv.get_coherence()
print(lda_umass)
print(lda_cv)


2021-07-18 01:42:42,641 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2021-07-18 01:42:42,649 : INFO : using WordOccurrenceAccumulator to estimate probabilities from sliding windows
2021-07-18 01:42:42,728 : INFO : WordOccurrenceAccumulator accumulated stats from 1000 documents


-4.58239006853846
0.29174151769917456


In [36]:
# import pyLDAvis.gensim
import pyLDAvis
import pyLDAvis.gensim_models
import pickle 

# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda, dtm, dictionary)
pyLDAvis.show(LDAvis_prepared)

# import pyLDAvis
# import pyLDAvis.gensim_models as gensimvis
# pyLDAvis.enable_notebook()

# lda_viz = gensimvis.prepare(lda, dtm, dictionary)
# pyLDAvis.show(LDAvis_prepared)

TypeError: ignored

In [37]:
##Label the topics based on representing "topic_words" obtained earlier
dict = {0: 'Love', 1: 'school', 2: 'young_crime', 3: 'politic', 4:'war'}

##Note that different runs result in different but simillar results if random_state is not specified


# Get the topic distribution of documents
doc_topics = lda.get_document_topics(dtm)
print(doc_topics)

In [38]:

from operator import itemgetter
#show the topic distributions for the first 5 docs, 
for i in range(0, 5):
    print(doc_topics[i]) # possible topics for each document
    print(max(doc_topics[i], key=itemgetter(1))[0]) # the highest possible topic

#Select the best topic (with highest score) for each document
top_topic = [ max(t, key=itemgetter(1))[0] for t in doc_topics ]
print (top_topic)




[(1, 0.25944185), (4, 0.7319132)]
4
[(0, 0.41027072), (1, 0.20912033), (3, 0.27004334), (4, 0.102745704)]
0
[(0, 0.11044), (1, 0.5343962), (3, 0.087849505), (4, 0.26399252)]
1
[(0, 0.9342147), (2, 0.05389428)]
0
[(0, 0.8989831), (1, 0.025124805), (2, 0.02571755), (3, 0.025066135), (4, 0.025108414)]
0
[4, 0, 1, 0, 0, 1, 4, 3, 3, 0, 0, 3, 1, 4, 1, 4, 2, 0, 4, 0, 3, 1, 0, 1, 4, 2, 1, 1, 0, 3, 2, 3, 1, 2, 4, 4, 1, 3, 3, 1, 3, 0, 1, 2, 0, 1, 4, 3, 1, 3, 4, 0, 3, 1, 3, 2, 3, 0, 0, 3, 3, 3, 4, 3, 1, 4, 2, 3, 0, 3, 3, 0, 2, 1, 3, 4, 1, 4, 4, 4, 1, 4, 1, 4, 4, 1, 4, 1, 4, 1, 4, 2, 2, 1, 4, 4, 1, 3, 1, 4, 4, 1, 4, 4, 2, 0, 1, 3, 3, 2, 3, 2, 3, 4, 2, 4, 4, 0, 1, 3, 1, 4, 2, 3, 1, 3, 4, 3, 1, 0, 1, 0, 3, 1, 2, 1, 3, 2, 0, 1, 3, 1, 1, 2, 4, 2, 1, 2, 3, 1, 4, 0, 4, 3, 4, 3, 2, 3, 1, 0, 4, 4, 2, 1, 3, 1, 3, 1, 1, 1, 3, 1, 2, 0, 4, 2, 3, 4, 0, 3, 0, 0, 1, 0, 1, 3, 2, 4, 1, 4, 3, 2, 1, 1, 4, 4, 4, 3, 1, 2, 3, 4, 3, 0, 2, 1, 0, 0, 2, 0, 3, 4, 1, 1, 3, 1, 4, 1, 4, 4, 3, 1, 4, 0, 4, 4, 2, 3, 4, 3, 2, 0, 1

In [39]:
topics_perDoc = [ dict[t] for t in top_topic ]
print(topics_perDoc)

['war', 'Love', 'school', 'Love', 'Love', 'school', 'war', 'politic', 'politic', 'Love', 'Love', 'politic', 'school', 'war', 'school', 'war', 'young_crime', 'Love', 'war', 'Love', 'politic', 'school', 'Love', 'school', 'war', 'young_crime', 'school', 'school', 'Love', 'politic', 'young_crime', 'politic', 'school', 'young_crime', 'war', 'war', 'school', 'politic', 'politic', 'school', 'politic', 'Love', 'school', 'young_crime', 'Love', 'school', 'war', 'politic', 'school', 'politic', 'war', 'Love', 'politic', 'school', 'politic', 'young_crime', 'politic', 'Love', 'Love', 'politic', 'politic', 'politic', 'war', 'politic', 'school', 'war', 'young_crime', 'politic', 'Love', 'politic', 'politic', 'Love', 'young_crime', 'school', 'politic', 'war', 'school', 'war', 'war', 'war', 'school', 'war', 'school', 'war', 'war', 'school', 'war', 'school', 'war', 'school', 'war', 'young_crime', 'young_crime', 'school', 'war', 'war', 'school', 'politic', 'school', 'war', 'war', 'school', 'war', 'war', 'y

In [40]:
####################################### How many dos in each topic?
labels, counts = np.unique(topics_perDoc, return_counts=True)
print (labels)
print (counts)

['Love' 'politic' 'school' 'war' 'young_crime']
[120 245 300 278 110]


In [ ]:
###########################Save and load pre-trained model
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("LDA_model")
lda.save(temp_file)
# Load a potentially pretrained model from disk.
lda = gensim.models.ldamodel.LdaModel.load(temp_file)

In [41]:
########################Query, the model using new, unseen documents
other_texts = [
    ['To','save','her','ailing','father','from','serving','in','the','Imperial','Army,','a','fearless','young','woman','disguises','herself','as','a','man','to','battle','northern','invaders','in','China'],#doc 1
    ['A','secret','agent','embarks','on','a','dangerous,','time-bending','mission','to','prevent','the','start','of','World','War','III'],#doc 2
]

new_dtm = [dictionary.doc2bow(d) for d in other_texts]

for i in range(0, 2):
    unseen_doc = new_dtm[i]
    print(lda[unseen_doc])

[(0, 0.022982748), (1, 0.023221293), (2, 0.20848148), (3, 0.022502275), (4, 0.72281224)]
[(0, 0.4730888), (1, 0.4402199), (2, 0.028582552), (3, 0.028722141), (4, 0.029386576)]


In [ ]:

########################Update the model by incrementally training on the new corpus
lda.update(new_dtm)


for i in range(0, 2):
    unseen_doc = new_dtm[i]
    print(lda[unseen_doc])
